In [0]:
from pyspark.sql import SparkSession, functions as F, types as T, Window
from delta.tables import DeltaTable
import os
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

In [0]:
from pyspark.sql import types as T

# Nome da tabela de destino no Catálogo Unity
catalog_table = "production.refined.d_forma_pagamentos"

# Verifica se a tabela já existe antes de tentar criá-la
if not spark.catalog.tableExists(catalog_table):
    print(f"A tabela {catalog_table} não existe. Criando a estrutura...")
    
    # Define o schema para a dimensão de formas de pagamento
    schema = T.StructType([
        # Chave primária (hash da chave de negócio: forma_pagamento)
        T.StructField("pk_forma_pagamento", T.StringType(), False),
        # Chave substituta (surrogate key) sequencial e única
        T.StructField("sk_forma_pagamento", T.LongType(), False),
        
        # Atributos descritivos da forma de pagamento
        T.StructField("forma_pagamento", T.StringType(), True),
        T.StructField("tipo_pagamento", T.StringType(), True), # Ex: 'Crédito', 'Débito', 'Transferência'

        # Metadados de controle (SCD - Slowly Changing Dimensions)
        T.StructField("start_date", T.DateType(), True),
        T.StructField("update_date", T.DateType(), True)
    ])
    
    # Cria um DataFrame vazio com o schema definido
    df_empty = spark.createDataFrame([], schema)

    # Cria a tabela Delta gerenciada no catálogo
    (
        df_empty.write
        .format("delta")
        .saveAsTable(catalog_table)
    )

    print(f"Tabela Delta '{catalog_table}' criada com sucesso.")
else:
    print(f"A tabela '{catalog_table}' já existe.")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from delta.tables import DeltaTable

# --- Parâmetros ---
catalog_table = "production.refined.d_forma_pagamentos"
# Assumindo que a origem das formas de pagamento também está na tabela de faturas
source_table = "production.trusted.tb_faturas_consumos" 

print("Iniciando processo de merge para a tabela:", catalog_table)

# --- 1. Carregar Tabela de Destino ---
try:
    delta_table = DeltaTable.forName(spark, catalog_table)
    df_dim_existente = delta_table.toDF()
    is_initial_load = df_dim_existente.count() == 0
except Exception as e:
    if "TABLE_OR_VIEW_NOT_FOUND" in str(e):
        is_initial_load = True
        print(f"Tabela {catalog_table} não encontrada. Assumindo carga inicial.")
    else:
        raise e

# --- 2. Carregar e Preparar Dados de Origem ---
# Obtém as formas de pagamento únicas da origem e calcula a chave primária (pk).
# ASSUNÇÃO: A tabela de origem possui as colunas 'forma_pagamento' e 'tipo_pagamento'.
# Se os nomes forem diferentes, ajuste-os no .select() abaixo.
df_source = (
    spark.read.table(source_table)
    .select("forma_pagamento", "tipo_pagamento")
    .filter(F.col("forma_pagamento").isNotNull())
    .dropDuplicates(["forma_pagamento"]) # A chave de negócio é o nome
    .withColumn(
        "pk_forma_pagamento",
        F.sha2(F.col("forma_pagamento"), 256)
    )
)

# --- 3. Identificar Apenas os Novos Registros ---
if is_initial_load:
    df_novas_formas_pagamento = df_source
else:
    # Usa um LEFT ANTI JOIN para isolar apenas os registros que NÃO existem no destino.
    df_novas_formas_pagamento = df_source.join(
        df_dim_existente.select("pk_forma_pagamento"),
        on="pk_forma_pagamento",
        how="left_anti"
    )

if not is_initial_load and df_novas_formas_pagamento.count() == 0:
    print("Nenhuma nova forma de pagamento para adicionar. Processo concluído.")
    # dbutils.notebook.exit("Nenhuma nova forma de pagamento para adicionar.")

# --- 4. Gerar Chaves Surrogadas (SKs) Apenas para os Novos Registros ---
if is_initial_load:
    last_id = 0
else:
    last_id_row = df_dim_existente.agg(F.max("sk_forma_pagamento")).collect()
    last_id = last_id_row[0][0] if last_id_row and last_id_row[0][0] is not None else 0

window = Window.orderBy("pk_forma_pagamento")
df_com_novas_sks = (
    df_novas_formas_pagamento
    .withColumn("sk_forma_pagamento", (F.row_number().over(window) + last_id).cast(T.LongType()))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast(T.DateType()))
)

# --- 5. Executar a Operação de MERGE (Apenas com Inserção) ---
# Seguindo o mesmo padrão de d_servicos, não há cláusula de UPDATE.
(
    delta_table.alias("target")
    .merge(
        df_com_novas_sks.alias("source"),
        "target.pk_forma_pagamento = source.pk_forma_pagamento"
    )
    .whenNotMatchedInsert(
        values={
            "pk_forma_pagamento": F.col("source.pk_forma_pagamento"),
            "sk_forma_pagamento": F.col("source.sk_forma_pagamento"),
            "forma_pagamento": F.col("source.forma_pagamento"),
            "tipo_pagamento": F.col("source.tipo_pagamento"),
            "start_date": F.col("source.start_date"),
            "update_date": F.col("source.update_date")
        }
    )
).execute()

print(f"Merge/upsert concluído com sucesso na tabela: {catalog_table}")

In [0]:
%sql
select * from production.refined.d_forma_pagamentos